# Datakaart

## Spoor 1
1. Lees NEDU profielen bestand in (nedu_files.h5)
2. Verwijder alle niet gebruikte profielen (E3, E4) en jaar 2021
3. Rol de kwartierdata op tot data per dag (optellen alle profielfactoren per kwartier)
4. Transformeer het dataframe om samenvoegen met verbruikdata mogelijk te maken op profiel:
    - Oud: (datum, E1A, E1B, E1C, E2A, E2B), bijv.:
        2010-01-01  0.003448  0.003514  0.003620  0.002332  0.003061
    - Nieuw: (datum, PROFIEL, FACTOR), bijv.:   
        2010-01-01  E1A  0.003448  
        2010-01-01  E1B  0.003514  
        2010-01-01  E1C  0.003620  
        2010-01-01  E2A  0.002332  
        2010-01-01  E2B  0.003061  
5. Vermenigvuldig de FACTOR kolom met 1e5 (stabiliteit model wordt dan beter)
5. Verdeel in train en test set op split datum '2018-01-01'
6. Creëer model en roep de fit functie aan op de train set
7. Bepaal de voorspelling op de test set en bereken de nauwkeurigheid (MAPE, RME, R2, )
8. Bepaal de voorspelling voor 2021-2023
9. Visualiseer de voorspellingen in 1 plot met meerdere kleuren mbv altair

## Spoor 2
1. Lees kleinverbruik bestand in (kleinverbruikgegevens_data.h5)
2. Voeg PROFIEL kolom toe obv soort aansluiting  
    * E1 profiel : 1x25, 3x25, 1x20 (met grote en kleine X in bestand), rest is E2
    * Obv laag tarief percentage < 50%, profiel A anders B
    * Als profiel E1 en B en postcodegebied (eerste 2 cijfers postcode) < 65 (zuiden) dan profiel C  
    Bijv:  
    SOORT_AANSLUITING = 1X25, SJV_LAAG_TARIEF_PERC = 60 en POSTCODE_AREA = 56 dan E1A  
    SOORT_AANSLUITING = 1x40, SJV_LAAG_TARIEF_PERC = 60 en POSTCODE_AREA = 89 dan E2B  
    SOORT_AANSLUITING = 3x25, SJV_LAAG_TARIEF_PERC = 40 en POSTCODE_AREA = 55 dan E1C  
3. Verdeel in train en test set op split datum '2018-01-01'
4. Creëer model en roep de fit functie aan op de train set
5. Bepaal de voorspelling op de test set en bereken de nauwkeurigheid (MAPE, RME, R2, )
6. Bepaal de voorspelling voor 2021-2023
7. Visualiseer de voorspellingen in 1 plot met meerdere kleuren mbv altair
8. Visualiseer de teruglevering in 1 plot met meerdere kleuren mbv altair

## Combinatie spoor
1. Voeg de profielen en kleinverbruik samen op de kolom PROFIEL (zorg er voor dat de kolommen DATUM, SJV_TOTAAL, FACTOR er in zitten)
2. Rol op tot PC4 VERBRUIK, waarbij VERBRUIK = SJV_TOTAAL * FACTOR
3. Lees bestand met PC4 en RES gegevens in (pc4_res.h5)
4. Rol op tot RES VERBRUIK
5. Creëer dataframe met de delta van jaar tot jaar per dag per RES

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import altair as alt

import nedu_prediction as nedu

# show all columns in the dataframe
pd.set_option('max_columns', None)

In [2]:
# Lees NEDU profielen bestand in (nedu_files.h5)
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# NEDU profielen
df_nedu_profielen = pd.read_hdf('nedu_files.h5')
df_nedu_profielen_origineel = df_nedu_profielen

# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [3]:
# Verwijder alle niet gebruikte profielen (E3, E4) 
drop_onnodige_profielen = {"E3A","E3B", 'E3C', 'E3D', 'E4A'}
df_nedu_profielen = df_nedu_profielen.drop(columns = drop_onnodige_profielen)
# Verwijder jaar 2021
df_nedu_profielen = df_nedu_profielen[df_nedu_profielen.DatumTijd < '2021-01-01']

In [4]:
# Rol de kwartierdata op tot data per dag (optellen alle profielfactoren per kwartier)
# feautures toevoegen zodat functie groupby werkt: jaar, maand, dag
df_nedu_profielen.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385727 entries, 0 to 385726
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   DatumTijd  385727 non-null  datetime64[ns]
 1   E1A        385727 non-null  float64       
 2   E1B        385727 non-null  float64       
 3   E1C        385727 non-null  float64       
 4   E2A        385727 non-null  float64       
 5   E2B        385727 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 20.6 MB


In [5]:
df_nedu_profielen['jaar'] = df_nedu_profielen.DatumTijd.dt.year
df_nedu_profielen['maand'] = df_nedu_profielen.DatumTijd.dt.month
df_nedu_profielen['dag'] = df_nedu_profielen.DatumTijd.dt.day

df_nedu_profielen = df_nedu_profielen.groupby(['jaar','maand','dag']).agg({'E1A':'sum', 'E1B':'sum', 'E1C':'sum', 'E2A':'sum', 'E2B':'sum'})

def maak_datum(jaar,maand,dag):
    return format(jaar,'04d') + '-' + format(maand,'02d') + '-' + format(dag,'02d')
df_nedu_profielen.index = np.vectorize(maak_datum)(df_nedu_profielen.index.get_level_values('jaar'), df_nedu_profielen.index.get_level_values('maand'), df_nedu_profielen.index.get_level_values('dag'))

In [6]:
# Verschillende dataframes maken voor de verschillende profielen
# op basis van df_nedu_profielen dataframe
df_nedu_e1a = pd.DataFrame(df_nedu_profielen['E1A']).reset_index().rename(columns={"index":"DATUM","E1A": "VERBRUIKS_FACTOR"})
df_nedu_e1b = pd.DataFrame(df_nedu_profielen['E1B']).reset_index().rename(columns={"index":"DATUM","E1B": "VERBRUIKS_FACTOR"})
df_nedu_e1c = pd.DataFrame(df_nedu_profielen['E1C']).reset_index().rename(columns={"index":"DATUM","E1C": "VERBRUIKS_FACTOR"})
df_nedu_e2a = pd.DataFrame(df_nedu_profielen['E2A']).reset_index().rename(columns={"index":"DATUM","E2A": "VERBRUIKS_FACTOR"})
df_nedu_e2b = pd.DataFrame(df_nedu_profielen['E2B']).reset_index().rename(columns={"index":"DATUM","E2B": "VERBRUIKS_FACTOR"})

# Vermenigvuldig de FACTOR kolom met 1e5 (stabiliteit model wordt dan beter)
df_nedu_e1a['VERBRUIKS_FACTOR'] = df_nedu_e1a['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e1b['VERBRUIKS_FACTOR'] = df_nedu_e1b['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e1c['VERBRUIKS_FACTOR'] = df_nedu_e1c['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e2a['VERBRUIKS_FACTOR'] = df_nedu_e2a['VERBRUIKS_FACTOR'] * 1e5
df_nedu_e2b['VERBRUIKS_FACTOR'] = df_nedu_e2b['VERBRUIKS_FACTOR'] * 1e5

In [7]:
# Prophet
# train model per profiel voor bepaalde train periode
from_date = '2010-01-01'
split_date = '2019-01-01'

nedu.train_model_nedu_profielen_prophet(df_nedu_e1a, profile='e1a', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e1b, profile='e1b', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e1c, profile='e1c', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e2a, profile='e2a', from_date=from_date, split_date=split_date)
nedu.train_model_nedu_profielen_prophet(df_nedu_e2b, profile='e2b', from_date=from_date, split_date=split_date)

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Profile e1a:
R^2   test: 0.94
MSE   test: 88.93
RMSE  test: 9.43
MAE   test: 7.20
MAPE  test: 2.59
MAPA  test: 97.41%

Model saved: NEDU_PRO_e1a_2019-01-01.pkl

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Profile e1b:
R^2   test: 0.89
MSE   test: 275.85
RMSE  test: 16.61
MAE   test: 14.31
MAPE  test: 5.30
MAPA  test: 94.70%

Model saved: NEDU_PRO_e1b_2019-01-01.pkl

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Profile e1c:
R^2   test: 0.91
MSE   test: 225.25
RMSE  test: 15.01
MAE   test: 12.87
MAPE  test: 4.78
MAPA  test: 95.22%

Model saved: NEDU_PRO_e1c_2019-01-01.pkl

Length full     dataset: 4018 samples
Length training dataset: 3288 samples
Length test     dataset: 730 samples

Measures for Pro

In [8]:
# maak een LOW prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'low'
df_nedu_e1a_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_low_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [9]:
# maak een MID prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'mid'
df_nedu_e1a_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_mid_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [10]:
# maak een HIGH prediction voor een profiel voor een bepaalde periode
predict_start = '2021-01-01'
predict_end   = '2023-12-31'
predict_type  = 'high'
df_nedu_e1a_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1b_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e1c_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e1c_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2a_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2a_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)
df_nedu_e2b_high_pred = nedu.predict_nedu_profielen_prophet(model='NEDU_PRO_e2b_'+split_date+'.pkl', start=predict_start, end=predict_end, predict_type=predict_type)

In [11]:
# DATUM aanpassen naar datetime formaat
df_nedu_e1a['DATUM'] = pd.to_datetime(df_nedu_e1a['DATUM'])
df_nedu_e1b['DATUM'] = pd.to_datetime(df_nedu_e1b['DATUM'])
df_nedu_e1c['DATUM'] = pd.to_datetime(df_nedu_e1c['DATUM'])
df_nedu_e2a['DATUM'] = pd.to_datetime(df_nedu_e2a['DATUM'])
df_nedu_e2b['DATUM'] = pd.to_datetime(df_nedu_e2b['DATUM'])

In [12]:
# Interactive zoom on both X and Y axis with two lower graphs.
df_nedu = df_nedu_e1a
df_nedu_pred_low = df_nedu_e1a_low_pred
df_nedu_pred_mid = df_nedu_e1a_mid_pred
df_nedu_pred_high = df_nedu_e1a_high_pred

interval = alt.selection_interval(encodings=['x'])
verbruik_range = alt.selection_interval(encodings=['y'])

base = alt.Chart(df_nedu.reset_index()).mark_line(clip=True).encode(
    x='DATUM:T',
    y='VERBRUIKS_FACTOR:Q'
)

forecast_low = alt.Chart(df_nedu_pred_low.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )
    
forecast_mid = alt.Chart(df_nedu_pred_mid.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )

forecast_high = alt.Chart(df_nedu_pred_high.reset_index()).mark_line(clip=True).encode(
                     x='DATUM:T',
                     y='VERBRUIKS_FACTOR:Q'
                     )
    
chart = base.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref()))
).properties(
    width=800,
    height=400)

forecast_low_chart = forecast_low.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('orange')
).properties(
    width=800,
    height=400)

forecast_mid_chart = forecast_mid.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('red')
).properties(
    width=800,
    height=400)

forecast_high_chart = forecast_high.encode(
    x=alt.X('DATUM:T', scale=alt.Scale(domain=interval.ref())),
    y=alt.Y('VERBRUIKS_FACTOR:Q', scale=alt.Scale(domain=verbruik_range.ref())),color=alt.value('orange')
).properties(
    width=800,
    height=400)
    
view = base+forecast_low.add_selection(
    interval
).properties(
    width=800,
    height=50,
)

window = base+forecast_low.add_selection(
    verbruik_range
).properties(
    width=800,
    height=100,
)

chart + forecast_low_chart + forecast_mid_chart + forecast_high_chart & view & window

alt.VConcatChart(...)

In [13]:
# MID prediction dataframes samenvoegen met bestaande NEDU dataframes
df_nedu_e1a_all = pd.concat([df_nedu_e1a[df_nedu_e1a['DATUM'] < predict_start], df_nedu_e1a_mid_pred])
df_nedu_e1b_all = pd.concat([df_nedu_e1b[df_nedu_e1b['DATUM'] < predict_start], df_nedu_e1b_mid_pred])
df_nedu_e1c_all = pd.concat([df_nedu_e1c[df_nedu_e1c['DATUM'] < predict_start], df_nedu_e1c_mid_pred])
df_nedu_e2a_all = pd.concat([df_nedu_e2a[df_nedu_e2a['DATUM'] < predict_start], df_nedu_e2a_mid_pred])
df_nedu_e2b_all = pd.concat([df_nedu_e2b[df_nedu_e2b['DATUM'] < predict_start], df_nedu_e2b_mid_pred])

In [14]:
# verbruiksfactor weer delen door 1e5
df_nedu_e1a_all['VERBRUIKS_FACTOR'] = df_nedu_e1a_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e1b_all['VERBRUIKS_FACTOR'] = df_nedu_e1b_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e1c_all['VERBRUIKS_FACTOR'] = df_nedu_e1c_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e2a_all['VERBRUIKS_FACTOR'] = df_nedu_e2a_all['VERBRUIKS_FACTOR'] / 1e5
df_nedu_e2b_all['VERBRUIKS_FACTOR'] = df_nedu_e2b_all['VERBRUIKS_FACTOR'] / 1e5

In [15]:
# alle profiel dataframes samenvoegen tot 1 dataframe
df_nedu_e1a_all['PROFIEL'] = "E1A"
df_nedu_e1b_all['PROFIEL'] = "E1B"
df_nedu_e1c_all['PROFIEL'] = "E1C"
df_nedu_e2a_all['PROFIEL'] = "E2A"
df_nedu_e2b_all['PROFIEL'] = "E2B"
df_nedu_all = pd.concat([df_nedu_e1a_all,df_nedu_e1b_all,df_nedu_e1c_all,df_nedu_e2a_all,df_nedu_e2b_all])

In [16]:
# jaar toevoegen aan nedu profielen om op te kunnen joinen met jaar verbruiksdata
df_nedu_all['JAAR'] = df_nedu_all['DATUM'].dt.year

In [17]:
df_nedu_all[df_nedu_all['PROFIEL'] == 'E1A'].tail()

,DATUM,VERBRUIKS_FACTOR,PROFIEL,JAAR
1090,2023-12-27,0.003353,E1A,2023
1091,2023-12-28,0.003340,E1A,2023
1092,2023-12-29,0.003376,E1A,2023
1093,2023-12-30,0.003434,E1A,2023
1094,2023-12-31,0.003434,E1A,2023


# Output van spoor 2 laden: verbruiksdata

In [18]:
#

# Samenvoeg spoor toevoegen

In [19]:
#